In [ ]:
# reloading magic
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patches as mpatches
import seaborn as sns
sns.set(style="white", palette="muted")
sns.set_context("talk", font_scale=1.4)
%matplotlib qt

In [ ]:
import matplotlib as mpl
mpl.rcParams['font.family'] = 'Helvetica'
mpl.rcParams['font.weight'] = 'light'
mpl.rcParams['axes.labelweight'] = 'light'


In [ ]:
# linux
comm_path = os.path.normpath('/home/oliver/repos/bikesim/comm')
scripts_path = os.path.normpath('/home/oliver/repos/bikesim/scripts')

In [ ]:
# osx
comm_path = os.path.normpath('/Users/oliverlee/repos/bikesim/comm')
scripts_path = os.path.normpath('/Users/oliverlee/repos/bikesim/scripts')

In [ ]:
for d in (comm_path, scripts_path):
    if d not in sys.path:
        sys.path.append(d)
from parse_data import *

In [ ]:
log_dirname = '/Users/oliverlee/repos/bikesim/comm/data/exp0'
if log_dirname is None:
    log_dirname = comm_path

In [ ]:
subject_map = parse_log_dir(log_dirname)

In [ ]:
fig, ax = plot_historgram(subject_map, 'torque')

In [ ]:
torques = []
deltas = []
deltads = []
for subj in subject_map.values():
    t = []
    d = []
    dd = []
    for log in subj.logs:
        t = np.append(t, log.get_field_in_timerange('torque'))
        d = np.append(d, log.get_field_in_timerange('delta'))
        dd = np.append(dd, log.get_field_in_timerange('deltad'))
    # ignore extra entries (time skew should be small)
    num_entries = min(len(t), len(d))
    torques.append(t[:num_entries])
    deltas.append(d[:num_entries])
    deltads.append(dd[:num_entries])

In [ ]:
peak = np.where(torques[0] > 10)[0]
counts = []
prev = 0
count = 0
for p in peak:
    if p - prev > 1:
        if count:
            counts.append(count)
        count = 0
    else:
        count += 1
    prev = p
print(counts)
print('avg: {} ms'.format(np.mean(counts) * 20))
print('max: {} ms'.format(max(counts) *  20))


In [ ]:
print([len(t) for t in torques])
print([len(dd) for dd in deltads])

In [ ]:
powers = []
for t, dd in zip(torques, deltads):
    powers.append(np.multiply(t, dd))
print('mean power: ', [np.mean(np.abs(p)) for p in powers])
print('std power: ', [np.std(p) for p in powers])

In [ ]:
subjects = list(subject_map.values())
print(subjects[0])

In [ ]:
for log in subjects[0].logs[:10]:
    y = log.get_field_in_timerange('torque')
    N = len(y)

    yf = np.fft.fft(y)
    xf = np.fft.fftfreq(len(y), 1/50)
    #xf = np.linspace(0.0, 1.0/(2.0*T), N/2)
    print(N/2)
    plt.semilogy(xf[1:N/2], 2.0/N * np.abs(yf[1:N/2]))


In [ ]:
inertia = 0.01 * 100**2 # kg m^2 * (100cm/1m)^2
print(inertia)

In [ ]:
# estimate handlebar inertia
# http://bikethomson.com/mtb-aluminum/
m_bar = 265/1000 # kg
h_bar = 750/1000 # m
r2_bar = 22.8 / 1000 / 2 # m, http://www.sheldonbrown.com/cribsheet-handlebars.html
r1_bar = 18.5 / 1000 / 2 # m, http://www.bikeforums.net/bicycle-mechanics/533030-handlebar-internal-diameter.html

# http://bikethomson.com/elite-x4-31-8-mountain/
m_stem = 141 / 1000 # kg
h_stem = 70 / 1000 # m, 70 mm stem length
r2_stem = 40.64 / 1000 / 2
r1_stem = 31.9 / 1000 / 2

inertia_bar = m_bar/12*(3*(r2_bar**2 - r1_bar**2) + h_bar**2) + m_bar*h_stem**2
inertia_stem = m_bar/12*(3*(r2_stem**2 - r1_stem**2) + h_stem**2) + m_stem*(h_stem/2)**2
print(inertia_bar)
print(inertia_stem)
print(inertia_bar + inertia_stem)

In [ ]:
fig, ax = plot_historgram(subject_map, 'deltad')

In [ ]:
df = balance_df(subject_map.values())
full_code = df.subject + '.' + df.torque_enabled.astype(int).astype(str)
df['full_subject_code'] = full_code
plt.subplots()
g = sns.boxplot(df.log_timespan, df.full_subject_code, color="Paired")
g.set_xlabel('subject')

In [ ]:
g1 = plot_dist_grouped_boxchart(subject_map)
plt.tight_layout()
plt.savefig('grouped_boxcharts.pdf', format='pdf', dpi=1000)
size = plt.gcf().get_size_inches()
# g1.fig.suptitle('Distribution of time balanced per subject', fontsize=18)

In [ ]:
fig2, ax2 = plot_dist_overlapping_histogram(subject_map)
plt.gcf().set_size_inches(size[0], size[1], forward=True)
plt.tight_layout()
plt.savefig('overlapping_histogram.pdf', format='pdf', dpi=1000)
# plt.subplots_adjust(top=0.95)
# fig2.suptitle('Distribution of time balanced for all subjects', fontsize=18)

In [ ]:
plt.close('all')
g3 = plot_subject_balance_time_change_boxplot(subject_map)
# plt.subplots_adjust(top=0.9)
plt.gcf().set_size_inches(size[0], size[1], forward=True)
plt.tight_layout()
plt.savefig('subject_change_boxplot.pdf', format='pdf', dpi=1000)
# g3.fig.suptitle('Change in time balanced per subject', fontsize=18)

In [ ]:
plt.close('all')
fig4, ax4 = plot_overlapping_psd(subject_map, 'phi', 'longest')
plt.gcf().set_size_inches(size[0], size[1], forward=True)
plt.tight_layout()
# plt.savefig('psd_lean_longest.pdf', format='pdf', dpi=1000)
# fig4.suptitle('Power spectral density of lean angle of longest run', fontsize=18)

In [ ]:
plt.close('all')
fig5, ax5 = plot_overlapping_psd(subject_map, 'phi', 'all')
plt.gcf().set_size_inches(size[0], size[1], forward=True)
plt.tight_layout()
plt.savefig('psd_lean_all.pdf', format='pdf', dpi=1000)
# fig5.suptitle('Power spectral density of lean angle of all runs', fontsize=18)

In [ ]:
fig6, ax6 = plot_overlapping_psd(subject_map, 'deltad', 'longest')
plt.gcf().set_size_inches(size[0], size[1], forward=True)
plt.tight_layout()
# plt.savefig('psd_steer_rate_longest.pdf', format='pdf', dpi=1000)
# fig6.suptitle('Power spectral density of steer rate of longest run', fontsize=18)

In [ ]:
plt.close('all')
fig7, ax7 = plot_overlapping_psd(subject_map, 'deltad', 'all')
plt.gcf().set_size_inches(size[0], size[1], forward=True)
plt.tight_layout()
plt.savefig('psd_steer_rate_all.pdf', format='pdf', dpi=1000)
# fig7.suptitle('Power spectral density of steer rate of all run', fontsize=18)

In [ ]:
plt.close('all')
fig8, ax8 = plot_overlapping_psd(subject_map, 'delta', 'all')
plt.gcf().set_size_inches(size[0], size[1], forward=True)
plt.tight_layout()
plt.savefig('psd_steer_all.pdf', format='pdf', dpi=1000)
# fig8.suptitle('Power spectral density of steer of all runs', fontsize=18)

In [ ]:
l = subject_map['003'].logs[-1]
print('feedback: {}'.format(l.feedback))
f, axes = plot_lean_steer_yy(l.sensor, l.actuator)
f, axes = plot_subplots(l.sensor, l.actuator, ('phi', 'delta', 'deltad'))

fig, ax = plt.subplots()
tr = l.timerange
ind_a = l.actuator.get_timerange_indices(tr)
ind_s = l.sensor.get_timerange_indices(tr)
f, psds = signal.welch(l.actuator.get_field('phi')[ind_a], 50, nperseg=256, return_onesided=True)
ax.loglog(f, psds, label='lean')
f, psds = signal.welch(l.sensor.get_field('delta')[ind_s], 50, nperseg=256, return_onesided=True)
ax.loglog(f, psds, label='steer')
f, psds = signal.welch(l.sensor.get_field('deltad')[ind_s], 50, nperseg=256, return_onesided=True)
ax.loglog(f, psds, label='steer rate')
ax.set_xlim([f[0], f[-1]])
ax.set_xlabel('frequency [Hz]')
ax.legend()
fig.suptitle('psd')

In [ ]:
for s in subject_map.values():
    t = s.balance_time(feedback=False)
    print(np.mean(t), np.std(t))